In [ ]:
# Código

import random
from river import tree
from river import datasets
from collections import defaultdict
import numpy as np

# Número de nodos en la red descentralizada
NUM_NODES = 3

# Dataset de ejemplo: Electricity
dataset = datasets.Elec2()

# Crear un Hoeffding Tree para cada nodo
nodes = [tree.HoeffdingTreeClassifier() for _ in range(NUM_NODES)]

# Estadísticas para la agregación
global_stats = defaultdict(lambda: defaultdict(float))  # {atributo: {valor: frecuencia}}

# Simulación de aprendizaje descentralizado
def simulate_federated_learning(dataset, nodes, num_rounds=10):
    # Dividir el dataset entre los nodos
    datasets_per_node = [[] for _ in range(NUM_NODES)]
    for x, y in dataset:
        node_id = random.randint(0, NUM_NODES - 1)  # Asignar ejemplos aleatoriamente a nodos
        datasets_per_node[node_id].append((x, y))

    # Iterar en rondas de aprendizaje descentralizado
    for round in range(num_rounds):
        print(f"\n--- Ronda {round + 1} ---")
        local_stats = [defaultdict(lambda: defaultdict(float)) for _ in range(NUM_NODES)]

        # Cada nodo entrena con sus datos locales y recopila estadísticas
        for node_id, node_data in enumerate(datasets_per_node):
            for x, y in node_data:
                nodes[node_id].learn_one(x, y)  # Entrenamiento local
                for feature, value in x.items():
                    local_stats[node_id][feature][value] += 1  # Actualizar frecuencias locales

        # Agregar estadísticas entre nodos
        for feature in local_stats[0]:
            for value in local_stats[0][feature]:
                global_stats[feature][value] = sum(local_stats[node_id][feature][value] for node_id in range(NUM_NODES)) / NUM_NODES

        print("Estadísticas globales actualizadas:")
        for feature, values in global_stats.items():
            print(f"  {feature}: {dict(values)}")

    # Evaluación final en datos globales
    evaluate_global_model(nodes)

# Evaluar los modelos combinados
def evaluate_global_model(nodes):
    print("\nEvaluación final combinada:")
    accuracy_per_node = []
    dataset = datasets.Elec2()
    correct_predictions = 0
    total_predictions = 0

    for x, y in dataset:
        # Votación mayoritaria entre nodos
        predictions = [node.predict_one(x) for node in nodes]
        prediction = max(set(predictions), key=predictions.count)  # Votación
        correct_predictions += prediction == y
        total_predictions += 1

    print(f"Precisión combinada: {correct_predictions / total_predictions:.2%}")

# Ejecutar la simulación
simulate_federated_learning(dataset, nodes, num_rounds=10)



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.

--- Ronda 1 ---
Estadísticas globales actualizadas:
  date: {0.0: 16.0, 4.4e-05: 16.0, 8.8e-05: 16.0, 0.000133: 16.0, 0.000177: 16.0, 0.000221: 16.0, 0.000265: 16.0, 0.00031: 16.0, 0.000354: 16.0, 0.000398: 16.0, 0.000442: 16.0, 0.000487: 16.0, 0.000531: 16.0, 0.000575: 16.0, 0.000619: 16.0, 0.000664: 16.0, 0.000708: 16.0, 0.000752: 16.0, 0.000796: 16.0, 0.000841: 16.0, 0.000885: 16.0, 0.000929: 16.0, 0.000973: 16.0, 0.001018: 16.0, 0.001062: 16.0, 0.004159: 16.0, 0.004203: 16.0, 0.004248: 16.0, 0.004292: 16.0, 0.004336: 16.0, 0.00438: 16.0, 0.004425: 16.0, 0.004469: 16.0, 0.004513: 16.0, 0.004557: 16.0, 0.004602: 16.0, 0.004646: 16.0, 0.00469: 16.0, 0.004734: 16.0, 0.004779: 16.0, 0.004823: 16.0, 0.004867: 16.0, 0.004911: 16.0, 0.004956: 16.0, 0.005: 16.0, 0.005044: 16.0, 0.005088: 16.0, 0.005133: 16.0, 0.005177: 16.0, 0.005221: 16.0, 0.00526

#### Cómo funciona el código

    - Creación de Hoeffding Trees:
        Cada nodo en el entorno descentralizado tiene su propio modelo local.

    - Distribución de Datos:
        El dataset se divide aleatoriamente entre los nodos.

    - Entrenamiento Local:
        Cada nodo entrena su modelo con su subconjunto de datos.

    - Recopilación de Estadísticas:
        Cada nodo calcula las frecuencias de las características en sus datos locales.
        Las estadísticas se agregan entre los nodos calculando promedios.

    - Actualización Global:
        Los modelos locales pueden usar estas estadísticas para decisiones de división más informadas (este paso puede ser extendido).

    - Evaluación Final:
        Los nodos realizan predicciones combinadas usando votación mayoritaria.

Weather:

    Descripción: Predicción del clima con drift gradual (temperatura, humedad, viento).
    Fuente: OpenML Weather Dataset.
